In [1]:
import os
import praw
import json
import datetime
from pprint import pprint


import praw.models
import praw.models.comment_forest
from dotenv import load_dotenv, find_dotenv

# Load environment variables
dotenv_path = find_dotenv()
if dotenv_path and os.getenv("REDDIT_CLIENT_ID") is None:
    load_dotenv(dotenv_path)

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
)


submission = reddit.submission(id="1gqscit")
submission.comments.replace_more(limit=5)

[]

In [2]:
def convert_unix_date(date: str) -> str:
    if date is False:
        return None
    return datetime.datetime.fromtimestamp(date, datetime.timezone.utc).strftime("%Y-%m-%d %H:%M:%S")

def serialize_comment(comments: praw.models.comment_forest.CommentForest) -> dict:
    serialized_comments = []
    for comment in comments:
        serialized_replies = serialize_comment(comment.replies)
        serialized_comments.append(
            {
                "author": str(comment.author),
                "body": comment.body,
                "comments": serialized_replies if serialized_replies else None,
                "created": convert_unix_date(comment.created_utc),
                "edited": convert_unix_date(comment.edited),
                "depth": comment.depth,
                "downs": comment.downs,
                "id": comment.id,
                "score": comment.score,
                "subreddit": comment.subreddit_name_prefixed,
                "ups": comment.ups,
            }
        )
    return serialized_comments

def serialize_submission(post: praw.models.Submission) -> dict:
    return {
        "author": str(post.author),
        "body": post.selftext,
        "comments": serialize_comment(post.comments),
        "created": convert_unix_date(post.created_utc),
        "edited": convert_unix_date(post.edited),
        "depth": -1,
        "downs": post.downs,
        "id": post.id,
        "num_comments": post.num_comments,  # not available in comments
        "score": post.score,
        "subreddit": post.subreddit_name_prefixed,
        "title": post.title,  # not available in comments
        "ups": post.ups,
        "url": post.url,  # not available in comments
    }

submission_dict = serialize_submission(submission)

pprint(submission_dict)

{'author': 'Flat_Singer4364',
 'body': 'Oh here we go another feminist upset because a man dared to explain '
         'basic biology. You know what the real issue is here it is not about '
         'being mansplained it is about the discomfort of having the truth '
         'laid out so plainly. The reality is that understanding calorie '
         'deficits and macros is fundamental to any serious discussion about '
         'fitness and health. It is not some sexist agenda it is literally how '
         'the body functions. If you want results you need to understand the '
         'science behind it.\n'
         '\n'
         'But no that is apparently offensive now right. We live in a time '
         'where explaining something factual is taken as a personal attack if '
         'it comes from a man. Instead of embracing the knowledge and '
         'recognizing that maybe the guy explaining it has years of experience '
         'and success in this area we get this knee-jerk reacti

In [3]:
submission.__dict__

{'comment_limit': 2048,
 'comment_sort': 'confidence',
 'id': '1gqscit',
 '_reddit': <praw.reddit.Reddit at 0x1142c91f0>,
 '_fetched': True,
 '_additional_fetch_params': {},
 '_comments_by_id': {'t1_lx0wodj': Comment(id='lx0wodj'),
  't1_lx0zynb': Comment(id='lx0zynb'),
  't1_lx0gpo8': Comment(id='lx0gpo8')},
 'approved_at_utc': None,
 'subreddit': Subreddit(display_name='copypasta'),
 'selftext': 'Oh here we go another feminist upset because a man dared to explain basic biology. You know what the real issue is here it is not about being mansplained it is about the discomfort of having the truth laid out so plainly. The reality is that understanding calorie deficits and macros is fundamental to any serious discussion about fitness and health. It is not some sexist agenda it is literally how the body functions. If you want results you need to understand the science behind it.\n\nBut no that is apparently offensive now right. We live in a time where explaining something factual is taken 

In [4]:
json.dumps(submission_dict)

'{"author": "Flat_Singer4364", "body": "Oh here we go another feminist upset because a man dared to explain basic biology. You know what the real issue is here it is not about being mansplained it is about the discomfort of having the truth laid out so plainly. The reality is that understanding calorie deficits and macros is fundamental to any serious discussion about fitness and health. It is not some sexist agenda it is literally how the body functions. If you want results you need to understand the science behind it.\\n\\nBut no that is apparently offensive now right. We live in a time where explaining something factual is taken as a personal attack if it comes from a man. Instead of embracing the knowledge and recognizing that maybe the guy explaining it has years of experience and success in this area we get this knee-jerk reaction about being oppressed or patronized.\\n\\nLet us look at the big picture. Trump won the presidency the Senate the House because people are fed up with 